# Importing LIbraries

In [28]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
#from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
#import folium # plotting library

!pip install beautifulsoup4
from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium
print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


# Importing data from wikipedia resource

In [29]:

url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

result = requests.get(url).text

In [30]:
soup = BeautifulSoup(result, 'lxml')

In [31]:
table = soup.find('table' , {'class':'wikitable sortable'})

In [32]:

tables = table.find_all('td')

In [33]:
col1 = []
col2 = []
col3 = []
i = 1
for link in tables:
    if i == 1:
        col1.append(link.text)
    elif i == 2:
        col2.append(link.text)
    else :
        col3.append(link.text)
    i = i + 1
    if i == 4:
        i = 1

In [34]:
raw_data = [ col1, col2 , col3]

# Converting data into a dataframe

In [35]:
raw_data = pd.DataFrame(raw_data)

In [36]:
raw_data= raw_data.transpose()

In [37]:
raw_data.rename(index = str, columns = {0:'Postcode', 1:'Borough', 2:'Neighborhood'},inplace = True)
raw_data.head()

,Postcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [38]:
#raw_data.Neighborhood.str.replace(r'\n','')
#raw_data.replace({'Neighborhood': '\n'}, {'Neighborhood': ''}, regex=True , inplace = True)
raw_data.Neighborhood = raw_data.Neighborhood.str.replace("\n","",regex = False)
raw_data.Postcode = raw_data.Postcode.str.replace("\n","",regex = False)
raw_data.Borough = raw_data.Borough.str.replace("\n","",regex = False)
raw_data.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [39]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180 entries, 0 to 179
Data columns (total 3 columns):
Postcode        180 non-null object
Borough         180 non-null object
Neighborhood    180 non-null object
dtypes: object(3)
memory usage: 5.6+ KB


In [40]:
#indexnames_col2 = raw_data[raw_data['Borough'] == 'Not assigned'].index
#raw_data.drop(index = indexnames_col2 , inplace = True)

indexnames_col3 = raw_data[raw_data['Neighborhood'] == 'Not assigned'].index
#raw_data.drop(index = indexnames_col3 , inplace = True)

#raw_data.reset_index(drop = True,inplace = True)

In [41]:

#len(raw_data.Neighborhood)
for index in range(len(raw_data.Neighborhood)):
    if raw_data.Neighborhood[index] == 'Not assigned':
        raw_data.Neighborhood[index] = raw_data.Borough[index]
raw_data.head(10)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [42]:
raw_data.groupby(["Postcode","Borough"])["Neighborhood"].apply(', '.join).reset_index()
raw_data.head(10)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


# Find and drop all "Not assigned" cells in Borough column

In [43]:
index_col2 = raw_data[raw_data['Borough'] == "Not assigned"].index

In [44]:
raw_data.drop(index = index_col2 , inplace = True)

In [45]:
raw_data.reset_index(drop = True, inplace = True)
raw_data.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


# Getting latitude and longtitude data for postal code and convert into dataframe

In [46]:
df_coordinate = pd.read_csv("Geospatial_Coordinates.csv")

In [47]:
df_coordinate.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


# Merge 2 tables into 1 with common postcode

In [48]:
df_coordinate.rename(index = str, columns = {'Postal Code':'Postcode'} , inplace = True)

In [49]:

df_merge = pd.merge(raw_data , df_coordinate, on = 'Postcode')

In [50]:
df_merge.head(20)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Display the neighborhoods on the map

In [54]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Showing the clustering of neighborhoods on the map....